In [1]:
%load_ext autoreload
%autoreload 2

# Compile mmcoremj.jar and copy dll and jar to Micro-Manager installation
import os
from pathlib import Path

os.chdir(r"C:\Users\henry\GitRepos\micro-manager\mmCoreAndDevices\MMCoreJ_wrap")
result = os.system("ant jar")
if result != 0:
    print(f"Error running 'ant jar': exit code {result}")

import shutil
source_1 = r"C:\Users\henry\GitRepos\micro-manager\mmCoreAndDevices\build\Debug\x64\MMCoreJ_wrap.dll"
destination_1 = r"C:\Program Files\Micro-Manager\MMCoreJ_wrap.dll"
shutil.copy2(source_1, destination_1)
source_2 = r"C:\Users\henry\GitRepos\micro-manager\build\Java\MMCoreJ.jar"
destination_2 = r"C:\Program Files\Micro-Manager\plugins\Micro-Manager\MMCoreJ.jar"
shutil.copy2(source_2, destination_2)
source_3 = r"C:\Users\henry\GitRepos\micro-manager\mmCoreAndDevices\build\Debug\x64\mmgr_dal_DemoCamera.dll"
destination_3 = r"C:\Program Files\Micro-Manager\mmgr_dal_DemoCamera.dll"
shutil.copy2(source_3, destination_3)
source_4 = r"C:\Users\henry\GitRepos\micro-manager\mmCoreAndDevices\build\Debug\x64\mmgr_dal_BaslerPylon.dll"
destination_4 = r"C:\Program Files\Micro-Manager\mmgr_dal_BaslerPylon.dll"
shutil.copy2(source_4, destination_4)

'C:\\Program Files\\Micro-Manager\\mmgr_dal_BaslerPylon.dll'

In [2]:
%load_ext autoreload
%autoreload 2

JAVA_BACKEND = True
config_file = 'MMConfig_basler.cfg'
# config_file = 'MMConfig_demo.cfg'


from pycromanager import start_headless, stop_headless, Core
import time
import numpy as np


if JAVA_BACKEND:
    def string_vec_to_list(sv):
        return [sv.get(i) for i in range(sv.size())]
    try:
        core = Core(convert_camel_case=False, timeout=1000)
    except Exception as e:
        start_headless('C:/Program Files/Micro-Manager', config_file, max_memory_mb=9000)
else:
    start_headless('C:/Program Files/Micro-Manager', config_file, python_backend=True)
    import pymmcore; print(pymmcore.__version__)

core = Core(convert_camel_case=False, timeout=5000)

In [5]:
# Test old core API (no triggering)

# Snap
assert not core.isSequenceRunning()
core.snapImage()
assert not core.isSequenceRunning()
image = core.getImage()
assert isinstance(image, np.ndarray)

# Continuous Sequence
assert not core.isSequenceRunning()
core.startContinuousSequenceAcquisition(0)
assert core.isSequenceRunning()
time.sleep(1)
core.stopSequenceAcquisition()
assert not core.isSequenceRunning()
ti = core.getLastTaggedImage()
assert isinstance(ti.pix, np.ndarray)

# Length 1 Sequence
assert not core.isSequenceRunning()
core.startSequenceAcquisition(1, 0, True)
assert core.isSequenceRunning()
while core.isSequenceRunning():
    time.sleep(0.1)
assert not core.isSequenceRunning()
ti = core.getLastTaggedImage()
assert isinstance(ti.pix, np.ndarray)


# Length N Sequence
assert not core.isSequenceRunning()
core.startSequenceAcquisition(10, 0, True)
assert core.isSequenceRunning()
while core.isSequenceRunning():
    time.sleep(0.1)
assert not core.isSequenceRunning()
for i in range(10):
    ti = core.popNextTaggedImage()
    assert isinstance(ti.pix, np.ndarray)


In [6]:
# Test new core API (no triggering)

# Length 1 Sequence
assert not core.isAcquisitionRunning()
core.acquisitionArm(1)
core.acquisitionStart()
assert core.isAcquisitionRunning()
while core.isAcquisitionRunning():
    time.sleep(0.1)
assert not core.isAcquisitionRunning()
ti = core.popNextTaggedImage()
assert isinstance(ti.pix, np.ndarray)

# Length N Sequence
assert not core.isAcquisitionRunning()
core.acquisitionArm(10)
core.acquisitionStart()
assert core.isAcquisitionRunning()
while core.isAcquisitionRunning():
    time.sleep(0.1)
assert not core.isAcquisitionRunning()
for i in range(10):
    ti = core.popNextTaggedImage()
    assert isinstance(ti.pix, np.ndarray)

# Continuous Sequence
assert not core.isAcquisitionRunning()
core.acquisitionArm(0)
core.acquisitionStart()
time.sleep(1)
core.acquisitionStop()
assert not core.isAcquisitionRunning()  
ti = core.getLastTaggedImage()
assert isinstance(ti.pix, np.ndarray)

# Continuous Sequence with abort instead of stop
assert not core.isAcquisitionRunning()
core.acquisitionArm(0)
core.acquisitionStart()
time.sleep(1)
core.acquisitionAbort()
assert not core.isAcquisitionRunning()
ti = core.getLastTaggedImage()
assert isinstance(ti.pix, np.ndarray)


## Triggering

In [3]:
import serial
import time

# Set up the serial connection (adjust COM port as needed)
arduino_port = "COM3"  # Change this to your Arduino's port (e.g., "/dev/ttyUSB0" for Linux/Mac)
baud_rate = 115200

try:
    ser = serial.Serial(arduino_port, baud_rate, timeout=1)
    time.sleep(2)  # Allow Arduino time to reset after opening serial
    print(f"Connected to Arduino on {arduino_port}")
except serial.SerialException:
    print(f"Error: Could not connect to {arduino_port}")
    exit()

def send_pulse(pin, duration):
    """Send a pulse command to Arduino."""
    if pin not in [1, 2]:
        print("Error: Invalid pin. Use 1 or 2.")
        return
    if duration <= 0:
        print("Error: Duration must be positive.")
        return

    command = f"{pin} {duration}\n"
    ser.write(command.encode())  # Send the command
    response = ser.readline().decode().strip()  # Read Arduino's response
    print(f"Arduino: {response}")


Connected to Arduino on COM3


In [4]:
# Test sending a trigger pulse
send_pulse(1, 100000)

Arduino: Pulse sent on pin 3 for 100000 us


In [5]:
# /* Continuous acquisition when the camera is in its reset state. */ 
# AcquisitionMode = Continuous;
#  AcquisitionStart();
#   ...
#    AcquisitionStop();

core.acquisitionArm(0)
assert not core.isAcquisitionRunning()
core.acquisitionStart()
assert core.isAcquisitionRunning()
time.sleep(1)
ti = core.getLastTaggedImage()
assert isinstance(ti.pix, np.ndarray)
core.acquisitionStop()
assert not core.isAcquisitionRunning()


In [10]:
#  /* Single Frame acquisition in Hardware trigger mode using the external I/O Line 3. */ 
#  AcquisitionMode = SingleFrame; 
#  TriggerSelector = FrameStart; 
#  TriggerMode = On; 
#  TriggerActivation = RisingEdge; 
#  TriggerSource = Line1; 
#  AcquisitionStart();

exposure = core.getExposure()
camera_device = core.getCameraDevice()
line = 1

core.clearCircularBuffer() # TODO: change to clearBuffer when new dataBuffer merges
core.acquisitionArm(1)
core.setProperty(camera_device, 'api//TriggerSelector', 'FrameStart')
core.setProperty(camera_device, 'api//TriggerMode', 'On')
core.setProperty(camera_device, 'api//TriggerActivation', 'RisingEdge')
core.setProperty(camera_device, 'api//TriggerSource', f'Line{line}')
core.acquisitionStart()
assert core.isAcquisitionRunning()
time.sleep(exposure / 1000 * 2)
# make sure nothing is in the buffer (i.e. acquired without a trigger)
try:
    core.popNextTaggedImage()
    assert False
except Exception as e:
    pass
send_pulse(line, 1000)
time.sleep(exposure / 1000 * 2)
ti = core.popNextTaggedImage()
assert isinstance(ti.pix, np.ndarray)
assert not core.isAcquisitionRunning()


Arduino: Pulse sent on pin 3 for 1000 us


In [18]:
# /* Multi-Frame acquisition started by a single Software trigger delayed by 1 millisecond.
#  The Trigger starts the whole sequence acquisition. The Exposure time for each frame is 
#  set to 500 us. */
# AcquisitionMode = MultiFrame; 
# AcquisitionFrameCount = 20; 
# TriggerSelector = AcquisitionStart; 
# TriggerMode = On; 
# TriggerSource = Software; 
# TriggerDelay = 1000; 
# ExposureMode = Timed; 
# ExposureTime = 500; 
# AcquisitionStart(); 
# TriggerSoftware();

frame_count = 20
exposure = .5

camera_device = core.getCameraDevice()

# check if camera has AcquisitionStartTrigger
test_valid = 'AcquisitionStart'  in string_vec_to_list(
    core.getAllowedPropertyValues(core.getCameraDevice(), 'api//TriggerSelector'))

if test_valid:
    core.clearCircularBuffer()  # TODO: change to clearBuffer when new dataBuffer merges
    core.acquisitionArm(frame_count) 
    core.setProperty(camera_device, 'api//TriggerSelector', 'AcquisitionStart')
    core.setProperty(camera_device, 'api//TriggerMode', 'On')
    core.setProperty(camera_device, 'api//TriggerSource', 'Software')
    core.setProperty(camera_device, 'api//TriggerDelay', 1000)  # 1ms delay
    core.setProperty(camera_device, 'api//ExposureMode', 'Timed')
    core.setProperty(camera_device, 'api//ExposureTime', int(exposure * 1000))  # 500us exposure

    # Start acquisition and trigger
    core.acquisitionStart()
    assert core.isAcquisitionRunning()
    time.sleep(exposure * 2)
    # assert nothing acquired without a trigger
    try:
        core.popNextTaggedImage()
        assert False
    except Exception as e:
        pass

    core.triggerCamera()  # Send a software trigger

    # Wait for all frames
    while core.isAcquisitionRunning():
        time.sleep(0.1)

    # Verify we got all 20 frames
    for i in range(frame_count):
        ti = core.popNextTaggedImage()
        assert isinstance(ti.pix, np.ndarray)
    # make sure nothing left
    try:
        core.popNextTaggedImage()
        assert False, "Buffer should be empty"
    except Exception:
        pass

    # assert the acquisition stopped
    assert not core.isAcquisitionRunning()


In [9]:
# /* Continuous acquisition in Hardware trigger mode. The Frame triggers are Rising Edge 
# signals coming from the physical Line 2. The Exposure time is 500us. An exposure
#  end event is also sent to the Host application after the exposure of each 
# frame to signal that the inspected part can be moved. The timestamp of the
#  event is also read. */ 
# AcquisitionMode = Continuous; 
# TriggerSelector = FrameStart; 
# TriggerMode = On; 
# TriggerActivation = RisingEdge; 
# TriggerSource = Line2; 
# ExposureMode = Timed; 
# ExposureTime = 500; 
# Register(Camera.EventExposureEnd, CallbackDataObject, CallbackFunctionPtr) EventSelector = ExposureEnd; 
# EventNotification = On; 

# Note: events are not guarenteed to be created, so its possible this test will fail

import threading

# Create event for tracking exposure end callback
exposure_end_event = threading.Event()


def callback_function(type, *args):
    if len(args) > 0 and args[0] == "ExposureEnd":
        exposure_end_event.set()

core.register_core_callback(callback_function)

camera_device = core.getCameraDevice()
core.setProperty(camera_device, 'api//TriggerMode', 'Off')
core.setProperty(camera_device, 'api//EventSelector', 'ExposureEnd')
core.setProperty(camera_device, 'api//EventNotification', 'On')


# Start acquisition and wait
core.acquisitionArm(0)
core.acquisitionStart()
time.sleep(2)
core.acquisitionStop()

# Verify we received the exposure end event
assert exposure_end_event.is_set(), "Never received ExposureEnd event"


In [11]:
# /* Continuous Acquisition of frames in bursts of 10 frames. Each burst is triggered by a
# Hardware trigger on Line 1. The end of each burst capture is signalled to the host with 
# a FrameBurstEnd event. */ 
# AcquisitionMode = Continuous; 
# AcquisitionBurstFrameCount = 10; 
# TriggerSelector = FrameBurstStart; 
# TriggerMode = On; 
# TriggerActivation = RisingEdge; 
# TriggerSource = Line1; 
# Register(Camera.EventFrameBurstEnd,CallbackDataObject,CallbackFunctionPtr) EventSelector = FrameBurstEnd; 
# EventNotification = On; 
# AcquisitionStart(); 
# ... 
# // In the callback of the end of burst event, get the event timestamp: 
# Timestamp = EventExposureEndTimestamp; 
# ... 
# AcquisitionStop();
# Note basler camera on which this is tested does not have a burst end event

images_per_burst = 10

camera_device = core.getCameraDevice()
exposure = core.getExposure()
core.setProperty(camera_device, 'api//AcquisitionBurstFrameCount', images_per_burst)
core.setProperty(camera_device, 'api//TriggerMode', 'On')
core.setProperty(camera_device, 'api//TriggerSelector', 'FrameBurstStart')
core.setProperty(camera_device, 'api//TriggerSource', 'Line1')
core.setProperty(camera_device, 'api//TriggerActivation', 'RisingEdge')


core.acquisitionArm(0)
core.acquisitionStart()

for i in range(2): # 2 bursts
    # trigger the burst
    send_pulse(1, 100000)
    # wait for 2x the total exposure time
    time.sleep(2 * images_per_burst * exposure / 1000)
    # read the images
    for i in range(images_per_burst):
        ti = core.popNextTaggedImage()
        assert isinstance(ti.pix, np.ndarray)
    # assert no more images
    try:
        core.popNextTaggedImage()
        assert False, "Buffer should be empty"
    except Exception:
        pass

core.acquisitionStop()  
assert not core.isAcquisitionRunning()


Arduino: Pulse sent on pin 3 for 100000 us
Arduino: Pulse sent on pin 3 for 100000 us


In [12]:
# /* Multi-Frame Acquisition of 50 frames in 5 bursts of 10 frames. Each burst is triggered by 
# a Hardware trigger on Line 1. */ 
# AcquisitionMode = MultiFrame; 
# AcquisitionFrameCount = 50; 
# AcquisitionBurstFrameCount = 10; 
# TriggerSelector = FrameBurstStart; 
# TriggerMode = On; 
# TriggerActivation = RisingEdge; 
# TriggerSource = Line1; 
# AcquisitionStart();

# /* Continuous Acquisition of frames in bursts of 10 frames. Each burst is triggered by a
# Hardware trigger on Line 1. The end of each burst capture is signalled to the host with 
# a FrameBurstEnd event. */ 
# AcquisitionMode = Continuous; 
# AcquisitionBurstFrameCount = 10; 
# TriggerSelector = FrameBurstStart; 
# TriggerMode = On; 
# TriggerActivation = RisingEdge; 
# TriggerSource = Line1; 
# Register(Camera.EventFrameBurstEnd,CallbackDataObject,CallbackFunctionPtr) EventSelector = FrameBurstEnd; 
# EventNotification = On; 
# AcquisitionStart(); 
# ... 
# // In the callback of the end of burst event, get the event timestamp: 
# Timestamp = EventExposureEndTimestamp; 
# ... 
# AcquisitionStop();
# Note basler camera on which this is tested does not have a burst end event


images_per_burst = 10
num_bursts = 5

camera_device = core.getCameraDevice()
exposure = core.getExposure()
core.setProperty(camera_device, 'api//AcquisitionBurstFrameCount', images_per_burst)
core.setProperty(camera_device, 'api//TriggerMode', 'On')
core.setProperty(camera_device, 'api//TriggerSelector', 'FrameBurstStart')
core.setProperty(camera_device, 'api//TriggerSource', 'Line1')
core.setProperty(camera_device, 'api//TriggerActivation', 'RisingEdge')


core.acquisitionArm(num_bursts * images_per_burst)
core.acquisitionStart()

for i in range(num_bursts): # 2 bursts
    # trigger the burst
    send_pulse(1, 100000)
    # wait for 2x the total exposure time
    time.sleep(2 * images_per_burst * exposure / 1000)
    # read the images
    for i in range(images_per_burst):
        ti = core.popNextTaggedImage()
        assert isinstance(ti.pix, np.ndarray)
    # assert no more images
    try:
        core.popNextTaggedImage()
        assert False, "Buffer should be empty"
    except Exception:
        pass

assert not core.isAcquisitionRunning()



Arduino: Pulse sent on pin 3 for 100000 us
Arduino: Pulse sent on pin 3 for 100000 us
Arduino: Pulse sent on pin 3 for 100000 us
Arduino: Pulse sent on pin 3 for 100000 us
Arduino: Pulse sent on pin 3 for 100000 us


In [ ]:
# /* Framescan continuous acquisition with Hardware Frame trigger and the Exposure duration
#  controlled by the Trigger pulse width. */ 
# AcquisitionMode = Continuous; 
# TriggerSelector = FrameStart; 
# TriggerMode = On; 
# TriggerActivation = RisingEdge; 
# TriggerSource = Line1; 
# ExposureMode = TriggerWidth; 
# AcquisitionStart(); 
# ... 
# AcquisitionStop();

# NOTE: this has not been tested on hardware yet


camera_device = core.getCameraDevice()
core.setProperty(camera_device, 'api//TriggerMode', 'On')
core.setProperty(camera_device, 'api//TriggerSelector', 'FrameStart')
core.setProperty(camera_device, 'api//TriggerSource', 'Line1')
core.setProperty(camera_device, 'api//TriggerActivation', 'RisingEdge')
core.setProperty(camera_device, 'api//ExposureMode', 'TriggerWidth')


core.acquisitionArm(0)
core.acquisitionStart()

# send a trigger pulse
send_pulse(1, 1000)
time.sleep(0.5)
pixels = core.popNextTaggedImage().pix
assert isinstance(pixels, np.ndarray)

# send another trigger pulse with longer exposure
send_pulse(1, 10000)
time.sleep(0.5)
pixels2 = core.popNextTaggedImage().pix
assert isinstance(pixels, np.ndarray)

core.acquisitionStop()

# the one with longer exposure should have greater intensity
assert pixels2.sum() > pixels.sum()
